In [7]:
# Required libraries
import pandas as pd
import csv
import sys
from collections import Counter
import matplotlib.pyplot as plt
import re, string, unicodedata
import nltk
from nltk import word_tokenize, sent_tokenize, FreqDist
from nltk.corpus import stopwords
import preprocessor as p
from ekphrasis.classes.segmenter import Segmenter
nltk.download
nltk.download('wordnet')
nltk.download('omw-1.4')
from wordcloud import WordCloud
import wandb
from wandb.keras import WandbCallback
import numpy as np
from keras.preprocessing import text
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from scipy.special import softmax
import random
import pickle
import time

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Jack\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Jack\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Jack\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [18]:
# loading data
data = pd.read_csv("C:\\Users\\Jack\\Desktop\\Uni Work\\PROJECT\\Data\\AirlinesData.csv")
data.drop(data.columns[[0,3,4,5,6,7,8,9,11,12,13,14]], axis=1, inplace=True)
data = data[data.airline_sentiment_confidence == 1.0] # removing rows that dont have 100% confidence 

In [79]:
data.head(10)

,airline_sentiment,airline_sentiment_confidence,text,Cleaned_Tweet
0,neutral,1.0,@VirginAmerica What @dhepburn said.,user What user said
3,negative,1.0,@VirginAmerica it's really aggressive to blast...,user its really aggressive to blast obnoxious ...
4,negative,1.0,@VirginAmerica and it's a really big bad thing...,user and its a really big bad thing about it
5,negative,1.0,@VirginAmerica seriously would pay $30 a fligh...,user seriously would pay 30 a flight for seats...
9,positive,1.0,"@VirginAmerica it was amazing, and arrived an ...",user it was amazing and arrived an hour early ...
11,positive,1.0,@VirginAmerica I &lt;3 pretty graphics. so muc...,user I lt3 pretty graphics so much better than...
12,positive,1.0,@VirginAmerica This is such a great deal! Alre...,user This is such a great deal Already thinkin...
14,positive,1.0,@VirginAmerica Thanks!,user Thanks
16,positive,1.0,@VirginAmerica So excited for my first cross c...,user So excited for my first cross country fli...
17,negative,1.0,@VirginAmerica I flew from NYC to SFO last we...,user I flew from NYC to SFO last week and coul...


In [71]:
# implementing placeholders
def preprocess(text):
    preprocessed_text = []
    for t in text.split():
        if len(t) > 1:
            t = '@user' if t[0] == '@' and t.count('@') == 1 else t
            t = 'http' if t.startswith('http') else t
            #t = '' if t.startswith('#') else t
        preprocessed_text.append(t)
    return ' '.join(preprocessed_text)


stop_words = set(stopwords.words('english'))



In [78]:
# pre-processing testing
Cleaned_Tweet =[]
for tweet in data["text"]:
    cleaned = preprocess(tweet)
    #tokenizedtweet = tweet.split()  # Tokenizing tweets
    #tweetNoStopWords = [word for word in tokenizedtweet if word not in stopwords.words('english')] # Removing stopwords
    #cleaned = ' '.join(tweetNoStopWords) # Joining tokenized tweet
    #cleaned = cleaned.translate(str.maketrans('', '', string.punctuation)) # Removing punctuation
    #cleaned = ''.join([i for i in cleaned if not i.isdigit()]) # removing digits
    Cleaned_Tweet.append(cleaned)
    
data["Cleaned_Tweet"] = Cleaned_Tweet
    


In [22]:
# loading model
roberta = "cardiffnlp/twitter-roberta-base-sentiment-latest"
model = AutoModelForSequenceClassification.from_pretrained(roberta)
tokenizer = AutoTokenizer.from_pretrained(roberta)
lables = ['negative', 'neutral', 'positive']


Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [80]:
# encoding tweets
encoded_tweets = []
for i in range(10445):
    encoded_tweet = tokenizer(Cleaned_Tweet[i], return_tensors='pt')
    encoded_tweets.append(encoded_tweet)
        
        

In [81]:
# executing sentiment analysis
All_scores = []
for i in range(10445):
    output = model(**encoded_tweets[i])
    print(i, "/10445")
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    scores
    All_scores.append(scores)
    

0 /10445
1 /10445
2 /10445
3 /10445
4 /10445
5 /10445
6 /10445
7 /10445
8 /10445
9 /10445
10 /10445
11 /10445
12 /10445
13 /10445
14 /10445
15 /10445
16 /10445
17 /10445
18 /10445
19 /10445
20 /10445
21 /10445
22 /10445
23 /10445
24 /10445
25 /10445
26 /10445
27 /10445
28 /10445
29 /10445
30 /10445
31 /10445
32 /10445
33 /10445
34 /10445
35 /10445
36 /10445
37 /10445
38 /10445
39 /10445
40 /10445
41 /10445
42 /10445
43 /10445
44 /10445
45 /10445
46 /10445
47 /10445
48 /10445
49 /10445
50 /10445
51 /10445
52 /10445
53 /10445
54 /10445
55 /10445
56 /10445
57 /10445
58 /10445
59 /10445
60 /10445
61 /10445
62 /10445
63 /10445
64 /10445
65 /10445
66 /10445
67 /10445
68 /10445
69 /10445
70 /10445
71 /10445
72 /10445
73 /10445
74 /10445
75 /10445
76 /10445
77 /10445
78 /10445
79 /10445
80 /10445
81 /10445
82 /10445
83 /10445
84 /10445
85 /10445
86 /10445
87 /10445
88 /10445
89 /10445
90 /10445
91 /10445
92 /10445
93 /10445
94 /10445
95 /10445
96 /10445
97 /10445
98 /10445
99 /10445
100 /10445

757 /10445
758 /10445
759 /10445
760 /10445
761 /10445
762 /10445
763 /10445
764 /10445
765 /10445
766 /10445
767 /10445
768 /10445
769 /10445
770 /10445
771 /10445
772 /10445
773 /10445
774 /10445
775 /10445
776 /10445
777 /10445
778 /10445
779 /10445
780 /10445
781 /10445
782 /10445
783 /10445
784 /10445
785 /10445
786 /10445
787 /10445
788 /10445
789 /10445
790 /10445
791 /10445
792 /10445
793 /10445
794 /10445
795 /10445
796 /10445
797 /10445
798 /10445
799 /10445
800 /10445
801 /10445
802 /10445
803 /10445
804 /10445
805 /10445
806 /10445
807 /10445
808 /10445
809 /10445
810 /10445
811 /10445
812 /10445
813 /10445
814 /10445
815 /10445
816 /10445
817 /10445
818 /10445
819 /10445
820 /10445
821 /10445
822 /10445
823 /10445
824 /10445
825 /10445
826 /10445
827 /10445
828 /10445
829 /10445
830 /10445
831 /10445
832 /10445
833 /10445
834 /10445
835 /10445
836 /10445
837 /10445
838 /10445
839 /10445
840 /10445
841 /10445
842 /10445
843 /10445
844 /10445
845 /10445
846 /10445
847 /10445

1462 /10445
1463 /10445
1464 /10445
1465 /10445
1466 /10445
1467 /10445
1468 /10445
1469 /10445
1470 /10445
1471 /10445
1472 /10445
1473 /10445
1474 /10445
1475 /10445
1476 /10445
1477 /10445
1478 /10445
1479 /10445
1480 /10445
1481 /10445
1482 /10445
1483 /10445
1484 /10445
1485 /10445
1486 /10445
1487 /10445
1488 /10445
1489 /10445
1490 /10445
1491 /10445
1492 /10445
1493 /10445
1494 /10445
1495 /10445
1496 /10445
1497 /10445
1498 /10445
1499 /10445
1500 /10445
1501 /10445
1502 /10445
1503 /10445
1504 /10445
1505 /10445
1506 /10445
1507 /10445
1508 /10445
1509 /10445
1510 /10445
1511 /10445
1512 /10445
1513 /10445
1514 /10445
1515 /10445
1516 /10445
1517 /10445
1518 /10445
1519 /10445
1520 /10445
1521 /10445
1522 /10445
1523 /10445
1524 /10445
1525 /10445
1526 /10445
1527 /10445
1528 /10445
1529 /10445
1530 /10445
1531 /10445
1532 /10445
1533 /10445
1534 /10445
1535 /10445
1536 /10445
1537 /10445
1538 /10445
1539 /10445
1540 /10445
1541 /10445
1542 /10445
1543 /10445
1544 /10445
1545

2146 /10445
2147 /10445
2148 /10445
2149 /10445
2150 /10445
2151 /10445
2152 /10445
2153 /10445
2154 /10445
2155 /10445
2156 /10445
2157 /10445
2158 /10445
2159 /10445
2160 /10445
2161 /10445
2162 /10445
2163 /10445
2164 /10445
2165 /10445
2166 /10445
2167 /10445
2168 /10445
2169 /10445
2170 /10445
2171 /10445
2172 /10445
2173 /10445
2174 /10445
2175 /10445
2176 /10445
2177 /10445
2178 /10445
2179 /10445
2180 /10445
2181 /10445
2182 /10445
2183 /10445
2184 /10445
2185 /10445
2186 /10445
2187 /10445
2188 /10445
2189 /10445
2190 /10445
2191 /10445
2192 /10445
2193 /10445
2194 /10445
2195 /10445
2196 /10445
2197 /10445
2198 /10445
2199 /10445
2200 /10445
2201 /10445
2202 /10445
2203 /10445
2204 /10445
2205 /10445
2206 /10445
2207 /10445
2208 /10445
2209 /10445
2210 /10445
2211 /10445
2212 /10445
2213 /10445
2214 /10445
2215 /10445
2216 /10445
2217 /10445
2218 /10445
2219 /10445
2220 /10445
2221 /10445
2222 /10445
2223 /10445
2224 /10445
2225 /10445
2226 /10445
2227 /10445
2228 /10445
2229

2829 /10445
2830 /10445
2831 /10445
2832 /10445
2833 /10445
2834 /10445
2835 /10445
2836 /10445
2837 /10445
2838 /10445
2839 /10445
2840 /10445
2841 /10445
2842 /10445
2843 /10445
2844 /10445
2845 /10445
2846 /10445
2847 /10445
2848 /10445
2849 /10445
2850 /10445
2851 /10445
2852 /10445
2853 /10445
2854 /10445
2855 /10445
2856 /10445
2857 /10445
2858 /10445
2859 /10445
2860 /10445
2861 /10445
2862 /10445
2863 /10445
2864 /10445
2865 /10445
2866 /10445
2867 /10445
2868 /10445
2869 /10445
2870 /10445
2871 /10445
2872 /10445
2873 /10445
2874 /10445
2875 /10445
2876 /10445
2877 /10445
2878 /10445
2879 /10445
2880 /10445
2881 /10445
2882 /10445
2883 /10445
2884 /10445
2885 /10445
2886 /10445
2887 /10445
2888 /10445
2889 /10445
2890 /10445
2891 /10445
2892 /10445
2893 /10445
2894 /10445
2895 /10445
2896 /10445
2897 /10445
2898 /10445
2899 /10445
2900 /10445
2901 /10445
2902 /10445
2903 /10445
2904 /10445
2905 /10445
2906 /10445
2907 /10445
2908 /10445
2909 /10445
2910 /10445
2911 /10445
2912

...

7619 /10445
7620 /10445
7621 /10445
7622 /10445
7623 /10445
7624 /10445
7625 /10445
7626 /10445
7627 /10445
7628 /10445
7629 /10445
7630 /10445
7631 /10445
7632 /10445
7633 /10445
7634 /10445
7635 /10445
7636 /10445
7637 /10445
7638 /10445
7639 /10445
7640 /10445
7641 /10445
7642 /10445
7643 /10445
7644 /10445
7645 /10445
7646 /10445
7647 /10445
7648 /10445
7649 /10445
7650 /10445
7651 /10445
7652 /10445
7653 /10445
7654 /10445
7655 /10445
7656 /10445
7657 /10445
7658 /10445
7659 /10445
7660 /10445
7661 /10445
7662 /10445
7663 /10445
7664 /10445
7665 /10445
7666 /10445
7667 /10445
7668 /10445
7669 /10445
7670 /10445
7671 /10445
7672 /10445
7673 /10445
7674 /10445
7675 /10445
7676 /10445
7677 /10445
7678 /10445
7679 /10445
7680 /10445
7681 /10445
7682 /10445
7683 /10445
7684 /10445
7685 /10445
7686 /10445
7687 /10445
7688 /10445
7689 /10445
7690 /10445
7691 /10445
7692 /10445
7693 /10445
7694 /10445
7695 /10445
7696 /10445
7697 /10445
7698 /10445
7699 /10445
7700 /10445
7701 /10445
7702

8303 /10445
8304 /10445
8305 /10445
8306 /10445
8307 /10445
8308 /10445
8309 /10445
8310 /10445
8311 /10445
8312 /10445
8313 /10445
8314 /10445
8315 /10445
8316 /10445
8317 /10445
8318 /10445
8319 /10445
8320 /10445
8321 /10445
8322 /10445
8323 /10445
8324 /10445
8325 /10445
8326 /10445
8327 /10445
8328 /10445
8329 /10445
8330 /10445
8331 /10445
8332 /10445
8333 /10445
8334 /10445
8335 /10445
8336 /10445
8337 /10445
8338 /10445
8339 /10445
8340 /10445
8341 /10445
8342 /10445
8343 /10445
8344 /10445
8345 /10445
8346 /10445
8347 /10445
8348 /10445
8349 /10445
8350 /10445
8351 /10445
8352 /10445
8353 /10445
8354 /10445
8355 /10445
8356 /10445
8357 /10445
8358 /10445
8359 /10445
8360 /10445
8361 /10445
8362 /10445
8363 /10445
8364 /10445
8365 /10445
8366 /10445
8367 /10445
8368 /10445
8369 /10445
8370 /10445
8371 /10445
8372 /10445
8373 /10445
8374 /10445
8375 /10445
8376 /10445
8377 /10445
8378 /10445
8379 /10445
8380 /10445
8381 /10445
8382 /10445
8383 /10445
8384 /10445
8385 /10445
8386

8987 /10445
8988 /10445
8989 /10445
8990 /10445
8991 /10445
8992 /10445
8993 /10445
8994 /10445
8995 /10445
8996 /10445
8997 /10445
8998 /10445
8999 /10445
9000 /10445
9001 /10445
9002 /10445
9003 /10445
9004 /10445
9005 /10445
9006 /10445
9007 /10445
9008 /10445
9009 /10445
9010 /10445
9011 /10445
9012 /10445
9013 /10445
9014 /10445
9015 /10445
9016 /10445
9017 /10445
9018 /10445
9019 /10445
9020 /10445
9021 /10445
9022 /10445
9023 /10445
9024 /10445
9025 /10445
9026 /10445
9027 /10445
9028 /10445
9029 /10445
9030 /10445
9031 /10445
9032 /10445
9033 /10445
9034 /10445
9035 /10445
9036 /10445
9037 /10445
9038 /10445
9039 /10445
9040 /10445
9041 /10445
9042 /10445
9043 /10445
9044 /10445
9045 /10445
9046 /10445
9047 /10445
9048 /10445
9049 /10445
9050 /10445
9051 /10445
9052 /10445
9053 /10445
9054 /10445
9055 /10445
9056 /10445
9057 /10445
9058 /10445
9059 /10445
9060 /10445
9061 /10445
9062 /10445
9063 /10445
9064 /10445
9065 /10445
9066 /10445
9067 /10445
9068 /10445
9069 /10445
9070

9671 /10445
9672 /10445
9673 /10445
9674 /10445
9675 /10445
9676 /10445
9677 /10445
9678 /10445
9679 /10445
9680 /10445
9681 /10445
9682 /10445
9683 /10445
9684 /10445
9685 /10445
9686 /10445
9687 /10445
9688 /10445
9689 /10445
9690 /10445
9691 /10445
9692 /10445
9693 /10445
9694 /10445
9695 /10445
9696 /10445
9697 /10445
9698 /10445
9699 /10445
9700 /10445
9701 /10445
9702 /10445
9703 /10445
9704 /10445
9705 /10445
9706 /10445
9707 /10445
9708 /10445
9709 /10445
9710 /10445
9711 /10445
9712 /10445
9713 /10445
9714 /10445
9715 /10445
9716 /10445
9717 /10445
9718 /10445
9719 /10445
9720 /10445
9721 /10445
9722 /10445
9723 /10445
9724 /10445
9725 /10445
9726 /10445
9727 /10445
9728 /10445
9729 /10445
9730 /10445
9731 /10445
9732 /10445
9733 /10445
9734 /10445
9735 /10445
9736 /10445
9737 /10445
9738 /10445
9739 /10445
9740 /10445
9741 /10445
9742 /10445
9743 /10445
9744 /10445
9745 /10445
9746 /10445
9747 /10445
9748 /10445
9749 /10445
9750 /10445
9751 /10445
9752 /10445
9753 /10445
9754

10328 /10445
10329 /10445
10330 /10445
10331 /10445
10332 /10445
10333 /10445
10334 /10445
10335 /10445
10336 /10445
10337 /10445
10338 /10445
10339 /10445
10340 /10445
10341 /10445
10342 /10445
10343 /10445
10344 /10445
10345 /10445
10346 /10445
10347 /10445
10348 /10445
10349 /10445
10350 /10445
10351 /10445
10352 /10445
10353 /10445
10354 /10445
10355 /10445
10356 /10445
10357 /10445
10358 /10445
10359 /10445
10360 /10445
10361 /10445
10362 /10445
10363 /10445
10364 /10445
10365 /10445
10366 /10445
10367 /10445
10368 /10445
10369 /10445
10370 /10445
10371 /10445
10372 /10445
10373 /10445
10374 /10445
10375 /10445
10376 /10445
10377 /10445
10378 /10445
10379 /10445
10380 /10445
10381 /10445
10382 /10445
10383 /10445
10384 /10445
10385 /10445
10386 /10445
10387 /10445
10388 /10445
10389 /10445
10390 /10445
10391 /10445
10392 /10445
10393 /10445
10394 /10445
10395 /10445
10396 /10445
10397 /10445
10398 /10445
10399 /10445
10400 /10445
10401 /10445
10402 /10445
10403 /10445
10404 /10445

In [82]:
Results = []
for i in range(len(All_scores)):
    max_value = np.amax(All_scores[i])
    max_label = lables[np.argmax(All_scores[i])]
    Results.append(max_label)

In [83]:
# printing accuracy
True_Values = data['airline_sentiment'].tolist()
matching = [i for i, j in zip(Results, True_Values) if i == j]
accuracy = ((len(matching)*100)/10445)
print("test accuracy is: ", accuracy)

test accuracy is:  82.80516993776926
